
# 🐢 Guardião das Tartaruguinhas: Monitoramento Comunitário de Ninhos

**Desenvolvido por:** Eric Narciso Pimentel dos Santos

**Curso:** Inteligência Artificial Aplicada aos Desafios Socioambientais da Amazônia

**Instituição:** I2A2 - Instituto de Inteligência Artificial Aplicada

**Data:** 28/07/2025

---

## 1. Contexto do Projeto

Em uma comunidade ribeirinha no coração da Amazônia, um esforço coletivo e inspirador de conservação dos quelônios ganha vida. Liderado por professores e jovens voluntários, o projeto **Guardião das Tartaruguinhas** visa monitorar os ninhos de tartarugas para garantir que o maior número possível de filhotes chegue com segurança ao rio, fortalecendo o ciclo de preservação e a cultura local.

## 2. O Desafio

Os dados dos ninhos — como localização, quantidade de ovos, status e riscos — eram registrados manualmente em papel. Essa metodologia, embora nobre, resultava em informações desorganizadas, inconsistentes e de difícil análise. Responder a perguntas essenciais para a tomada de decisão, como "Qual a média de ovos?" ou "Qual região apresenta maior risco?", tornava-se uma tarefa árdua e imprecisa.

## 3. A Solução Proposta

Este notebook representa a solução tecnológica para o desafio. Trata-se de um sistema simples, porém robusto, desenvolvido em Python para permitir que os voluntários da comunidade possam:

* **Registrar** os dados dos ninhos de forma padronizada.
* **Validar** as informações no momento da entrada para garantir sua integridade.
* **Analisar** os dados acumulados para gerar estatísticas e insights acionáveis.
* **Apoiar** a tomada de decisão semanal do grupo de proteção, otimizando os esforços de conservação.

> ## Nosso desafio não é apenas contar os ovos. É usar cada número, cada dado, para tecer uma rede de proteção em torno da vida. Aqui, cada linha de código é um ato de esperança.

Este projeto é a ponte entre o conhecimento tradicional e a tecnologia acessível, transformando dados em um escudo para a biodiversidade amazônica.



In [ ]:
# ----------------------------------------------------------------------------
# SEÇÃO 0: CONSTANTES E CONFIGURAÇÕES
# ----------------------------------------------------------------------------

# --- Regras de Negócio ---
OVOS_MIN_REALISTA = 5
OVOS_MAX_REALISTA = 200
DIAS_ECLOSAO_MIN = 1
DIAS_ECLOSAO_MAX = 130

# --- Constantes Visuais (Códigos de Escape ANSI para Cores) ---
CORES = {
    "VERDE": "\033[92m",
    "AMARELO": "\033[93m",
    "VERMELHO": "\033[91m",
    "AZUL": "\033[94m",
    "CIANO": "\033[96m",
    "NEGRITO": "\033[1m",
    "RESET": "\033[0m"
}

# ----------------------------------------------------------------------------
# SEÇÃO 1: ESTRUTURA DE DADOS E CARGA INICIAL (ENRIQUECIDA)
# ----------------------------------------------------------------------------
ninhos = [
    {"regiao": "Praia Norte", "quantidade_ovos": 102, "status": "intacto", "risco": "🟢", "dias_para_eclosao": 12, "predadores": False, "causa_risco": "nenhuma"},
    {"regiao": "Praia Central", "quantidade_ovos": 89, "status": "danificado", "risco": "🔴", "dias_para_eclosao": 3, "predadores": True, "causa_risco": "predador"},
    {"regiao": "Praia Sul", "quantidade_ovos": 120, "status": "ameaçado", "risco": "🟡", "dias_para_eclosao": 7, "predadores": False, "causa_risco": "alagamento"},
    {"regiao": "Praia Central", "quantidade_ovos": 75, "status": "intacto", "risco": "🟢", "dias_para_eclosao": 2, "predadores": False, "causa_risco": "nenhuma"},
    {"regiao": "Praia Norte", "quantidade_ovos": 60, "status": "danificado", "risco": "🔴", "dias_para_eclosao": 5, "predadores": True, "causa_risco": "predador"},
    {"regiao": "Praia Sul", "quantidade_ovos": 95, "status": "intacto", "risco": "🟢", "dias_para_eclosao": 20, "predadores": False, "causa_risco": "nenhuma"},
    {"regiao": "Praia Norte", "quantidade_ovos": 110, "status": "ameaçado", "risco": "🟡", "dias_para_eclosao": 8, "predadores": True, "causa_risco": "predador"},
    {"regiao": "Praia Central", "quantidade_ovos": 81, "status": "ameaçado", "risco": "🟡", "dias_para_eclosao": 15, "predadores": False, "causa_risco": "humano"},
    {"regiao": "Praia Sul", "quantidade_ovos": 70, "status": "danificado", "risco": "🔴", "dias_para_eclosao": 1, "predadores": True, "causa_risco": "predador"},
    {"regiao": "Praia Norte", "quantidade_ovos": 130, "status": "intacto", "risco": "🟢", "dias_para_eclosao": 25, "predadores": False, "causa_risco": "nenhuma"},
]

# ----------------------------------------------------------------------------
# SEÇÃO 2: FUNÇÕES UTILITÁRIAS E DE DESIGN
# ----------------------------------------------------------------------------

def desenhar_divisor(caractere="~", tamanho=80):
    """Cria um divisor temático para a interface."""
    print(f"{CORES['AZUL']}{caractere * tamanho}{CORES['RESET']}")

def gerar_barra_percentual(percentual, largura=20):
    """Cria uma barra de progresso textual para visualização."""
    blocos_preenchidos = int((percentual / 100) * largura)
    barra = "█" * blocos_preenchidos + " " * (largura - blocos_preenchidos)
    return f"[{barra}]"

# ----------------------------------------------------------------------------
# SEÇÃO 3: FUNÇÕES DE ANÁLISE E RELATÓRIOS
# ----------------------------------------------------------------------------

def exibir_painel_alerta(base_de_ninhos):
    """Mostra um dashboard com os indicadores mais críticos."""
    desenhar_divisor("=")
    print("||" + f"{CORES['NEGRITO']}{CORES['VERMELHO']} 🚨 PAINEL DE ALERTA RÁPIDO 🚨 {CORES['RESET']}".center(96) + "||")
    desenhar_divisor("=")
    if not base_de_ninhos: print("\nNenhum dado para análise.\n"); desenhar_divisor("="); return

    ninhos_risco_critico = sum(1 for n in base_de_ninhos if n["risco"] == "🔴")
    ninhos_eclosao_urgente = sum(1 for n in base_de_ninhos if n["dias_para_eclosao"] <= 2)

    contagem_critica_regiao = {}
    for n in base_de_ninhos:
        if n["risco"] == "🔴":
            regiao = n["regiao"]
            contagem_critica_regiao[regiao] = contagem_critica_regiao.get(regiao, 0) + 1

    regiao_mais_critica = "Nenhuma"
    if contagem_critica_regiao:
        regiao_mais_critica = max(contagem_critica_regiao, key=contagem_critica_regiao.get)

    total_ninhos = len(base_de_ninhos)
    ninhos_intactos = sum(1 for n in base_de_ninhos if n["status"] == "intacto")
    taxa_sobrevivencia = (ninhos_intactos / total_ninhos) * 100 if total_ninhos > 0 else 0

    print(f"\n  - {CORES['VERMELHO']}Ninhos em Risco Crítico (🔴):{CORES['RESET']} {ninhos_risco_critico} ninhos")
    print(f"  - {CORES['CIANO']}Eclosão Urgente (em até 48h):{CORES['RESET']} {ninhos_eclosao_urgente} ninhos")
    print(f"  - {CORES['AMARELO']}Região com Maior Risco Crítico:{CORES['RESET']} {regiao_mais_critica}")
    print(f"  - {CORES['VERDE']}Taxa de Ninhos Intactos:{CORES['RESET']} {taxa_sobrevivencia:.1f}%")
    print()
    desenhar_divisor("=")

def exibir_relatorio_completo(base_de_ninhos):
    """Exibe um relatório formatado e colorido de todos os ninhos."""
    desenhar_divisor()
    print("||" + " RELATÓRIO COMPLETO DE NINHOS ".center(76) + "||")
    desenhar_divisor()
    if not base_de_ninhos:
        print("\nNenhum ninho registrado na base de dados.\n")
    else:
        print(f"| {CORES['NEGRITO']}{'ID':<3} | {'REGIÃO':<15} | {'OVOS':<5} | {'STATUS':<12} | {'RISCO':<7} | {'ECLOSÃO (dias)':<16} | {'PREDADORES':<10} | {'CAUSA RISCO':<12}{CORES['RESET']} |")
        print("-" * 95)
        id_ninho = 1
        for ninho in base_de_ninhos:
            cor = CORES["RESET"]
            if ninho["risco"] == "🟢": cor = CORES["VERDE"]
            elif ninho["risco"] == "🟡": cor = CORES["AMARELO"]
            elif ninho["risco"] == "🔴": cor = CORES["VERMELHO"]

            predadores_str = "Sim" if ninho["predadores"] else "Não"
            print(f"{cor}| {id_ninho:<3} | {ninho['regiao']:<15} | {ninho['quantidade_ovos']:<5} | {ninho['status']:<12} | {ninho['risco']:<7} | {ninho['dias_para_eclosao']:<16} | {predadores_str:<10} | {ninho['causa_risco']:<12} |{CORES['RESET']}")
            id_ninho += 1
    desenhar_divisor()

def calcular_estatisticas_basicas(base_de_ninhos):
    """Calcula e exibe as estatísticas básicas de forma visual."""
    desenhar_divisor()
    print("||" + " ESTATÍSTICAS BÁSICAS DE MONITORAMENTO ".center(76) + "||")
    desenhar_divisor()
    if not base_de_ninhos: print("\nNenhum dado para análise.\n"); desenhar_divisor(); return

    total_ninhos = len(base_de_ninhos)
    print(f"\n[ 1 ] TOTAL DE NINHOS: {CORES['AZUL']}{CORES['NEGRITO']}{total_ninhos}{CORES['RESET']}")

    ovos_estaveis, ninhos_estaveis = 0, 0
    for n in base_de_ninhos:
        if n["risco"] == "🟢": ovos_estaveis += n["quantidade_ovos"]; ninhos_estaveis += 1
    if ninhos_estaveis > 0: print(f"[ 2 ] MÉDIA DE OVOS (ESTÁVEIS 🟢): {CORES['VERDE']}{int(ovos_estaveis/ninhos_estaveis)}{CORES['RESET']} ovos")
    else: print("[ 2 ] MÉDIA DE OVOS (ESTÁVEIS �): Não há ninhos estáveis.")

    ninhos_eclosao = sum(1 for n in base_de_ninhos if n["dias_para_eclosao"] <= 5)
    print(f"[ 3 ] ECLOSÃO PRÓXIMA (<= 5 dias): {CORES['CIANO']}{ninhos_eclosao}{CORES['RESET']} ninhos")

    risco_regiao = {}
    for n in base_de_ninhos:
        if n["risco"] == "🟡":
            risco_regiao[n["regiao"]] = risco_regiao.get(n["regiao"], 0) + 1

    if not risco_regiao: print("[ 4 ] REGIÃO COM MAIS RISCO (🟡): Nenhuma.")
    else: regiao_mais_risco = max(risco_regiao, key=risco_regiao.get); print(f"[ 4 ] REGIÃO COM MAIS RISCO (🟡): {CORES['AMARELO']}{regiao_mais_risco} ({risco_regiao[regiao_mais_risco]} ninhos){CORES['RESET']}")

    danificados_predadores = sum(1 for n in base_de_ninhos if n["status"] == "danificado" and n["predadores"])
    print(f"[ 5 ] DANIFICADOS C/ PREDADORES: {CORES['VERMELHO']}{danificados_predadores}{CORES['RESET']} ninhos")
    print()
    desenhar_divisor()

def analise_descritiva_completa(base_de_ninhos):
    """Fornece um panorama estatístico descritivo com visualizações."""
    desenhar_divisor()
    print("||" + " ANÁLISE DESCRITIVA COMPLETA ".center(76) + "||")
    desenhar_divisor()
    if not base_de_ninhos: print("\nNenhum dado para análise.\n"); desenhar_divisor(); return

    print(f"\n--- 1. Análise de Ovos por Status do Ninho ---")
    stats_ovos = {"intacto": [], "ameaçado": [], "danificado": []}
    for n in base_de_ninhos: stats_ovos[n["status"]].append(n["quantidade_ovos"])
    for status, ovos_lista in stats_ovos.items():
        if ovos_lista:
            media = int(sum(ovos_lista) / len(ovos_lista))
            print(f"  - {status.title():<12}: Média={media:<4} Mín={min(ovos_lista):<4} Máx={max(ovos_lista):<4}")
        else:
            print(f"  - {status.title():<12}: Nenhum ninho encontrado.")

    print("\n--- 2. Distribuição Percentual dos Ninhos ---")
    total = len(base_de_ninhos)
    dist_status = {s: sum(1 for n in base_de_ninhos if n['status'] == s) for s in ["intacto", "ameaçado", "danificado"]}
    dist_risco = {r: sum(1 for n in base_de_ninhos if n['risco'] == r) for r in ["🟢", "🟡", "🔴"]}

    print("  - Por Status:")
    for status, count in dist_status.items():
        percentual = (count / total) * 100
        barra = gerar_barra_percentual(percentual)
        print(f"    {status.title():<12} {barra} {percentual:.1f}% ({count}/{total})")
    print("  - Por Risco:")
    for risco, count in dist_risco.items():
        percentual = (count / total) * 100
        barra = gerar_barra_percentual(percentual)
        print(f"    Risco {risco}      {barra} {percentual:.1f}% ({count}/{total})")

    print("\n--- 3. Ranking de Regiões (Mapa de Calor de Risco) ---")
    contagem_regiao = {}
    for n in base_de_ninhos:
        contagem_regiao[n["regiao"]] = contagem_regiao.get(n["regiao"], 0) + 1

    regioes_ordenadas = sorted(contagem_regiao.items(), key=lambda item: item[1], reverse=True)
    for i, (regiao, count) in enumerate(regioes_ordenadas):
        ninhos_na_regiao = [n for n in base_de_ninhos if n['regiao'] == regiao]
        risco_critico = sum(1 for n in ninhos_na_regiao if n['risco'] == '🔴')

        cor_regiao = CORES['VERDE']
        if risco_critico > 0:
            taxa_risco = (risco_critico / len(ninhos_na_regiao)) * 100
            if taxa_risco > 50:
                cor_regiao = CORES['VERMELHO']
            else:
                cor_regiao = CORES['AMARELO']

        print(f"  {i+1}º. {cor_regiao}{regiao:<15}{CORES['RESET']}: {count} ninhos")
    print()
    desenhar_divisor()

def analise_diagnostica_cruzada(base_de_ninhos):
    """Busca responder 'por quês' ao cruzar diferentes variáveis."""
    desenhar_divisor()
    print("||" + " ANÁLISE DIAGNÓSTICA CRUZADA ".center(76) + "||")
    desenhar_divisor()
    if not base_de_ninhos: print("\nNenhum dado para análise.\n"); desenhar_divisor(); return

    print("\n--- 1. Diagnóstico de Causa para Ninhos Danificados ---")
    ninhos_danificados = [n for n in base_de_ninhos if n['status'] == 'danificado']
    if not ninhos_danificados:
        print("  Nenhum ninho danificado para analisar.")
    else:
        causas_danos = {}
        for n in ninhos_danificados:
            causas_danos[n["causa_risco"]] = causas_danos.get(n["causa_risco"], 0) + 1

        causa_principal = max(causas_danos, key=causas_danos.get)
        print(f"  A principal causa de danos é: {CORES['VERMELHO']}'{causa_principal.title()}'{CORES['RESET']} ({causas_danos[causa_principal]}/{len(ninhos_danificados)} casos)")

    print("\n--- 2. Análise de Impacto de Predadores ---")
    ninhos_com_predador = [n for n in base_de_ninhos if n['predadores']]
    if not ninhos_com_predador:
        print("  Nenhum ninho com registro de predadores.")
    else:
        danificados = sum(1 for n in ninhos_com_predador if n['status'] == 'danificado')
        taxa_sucesso = (danificados / len(ninhos_com_predador)) * 100
        cor_taxa = CORES['VERMELHO'] if taxa_sucesso >= 50 else CORES['AMARELO']
        print(f"  Dos {len(ninhos_com_predador)} ninhos com predadores, {danificados} foram danificados.")
        print(f"  Taxa de sucesso da predação: {cor_taxa}{taxa_sucesso:.1f}%{CORES['RESET']}")
    print()
    desenhar_divisor()

def adicionar_novo_ninho(base_de_ninhos):
    """Guia o usuário para adicionar um novo registro de ninho, com validação robusta."""
    desenhar_divisor("-")
    print("||" + " ADICIONAR NOVO REGISTRO DE NINHO ".center(76) + "||")
    desenhar_divisor("-")

    # Validação da Região
    regioes_conhecidas = sorted(list(set(n["regiao"] for n in base_de_ninhos)))
    while True:
        print(f"Regiões conhecidas: {', '.join(regioes_conhecidas)}")
        regiao_input = input("Digite o nome da região: ").strip().title()
        if regiao_input in regioes_conhecidas:
            regiao = regiao_input
            break
        elif regiao_input:
            confirmacao = input(f"A região '{regiao_input}' é nova. Deseja adicioná-la? (s/n): ").lower().strip()
            if confirmacao == 's':
                regiao = regiao_input
                break
        else:
            print(f"{CORES['VERMELHO']}Erro: O nome da região não pode ser vazio.{CORES['RESET']}")

    # Validação da Quantidade de Ovos
    while True:
        ovos_input = input(f"Digite a quantidade de ovos ({OVOS_MIN_REALISTA}-{OVOS_MAX_REALISTA}): ").strip()
        if ovos_input.isdigit():
            quantidade_ovos = int(ovos_input)
            if OVOS_MIN_REALISTA <= quantidade_ovos <= OVOS_MAX_REALISTA:
                break
            else:
                print(f"{CORES['VERMELHO']}Erro: Valor fora dos limites realistas.{CORES['RESET']}")
        else:
            print(f"{CORES['VERMELHO']}Erro: Por favor, digite um número inteiro válido.{CORES['RESET']}")

    # Validação do Status
    status_validos = ["intacto", "ameaçado", "danificado"]
    while True:
        status = input(f"Digite o status ({'/'.join(status_validos)}): ").lower().strip()
        if status in status_validos:
            break
        else:
            print(f"{CORES['VERMELHO']}Erro: Status inválido.{CORES['RESET']}")

    # Validação do Risco
    riscos_validos = ["🟢", "🟡", "🔴"]
    while True:
        risco = input(f"Digite o risco ({''.join(riscos_validos)}): ").strip()
        if risco in riscos_validos:
            break
        else:
            print(f"{CORES['VERMELHO']}Erro: Risco inválido.{CORES['RESET']}")

    # Validação dos Dias para Eclosão
    while True:
        dias_input = input(f"Digite os dias para eclosão ({DIAS_ECLOSAO_MIN}-{DIAS_ECLOSAO_MAX}): ").strip()
        if dias_input.isdigit():
            dias_para_eclosao = int(dias_input)
            if DIAS_ECLOSAO_MIN <= dias_para_eclosao <= DIAS_ECLOSAO_MAX:
                break
            else:
                print(f"{CORES['VERMELHO']}Erro: Valor fora dos limites realistas.{CORES['RESET']}")
        else:
            print(f"{CORES['VERMELHO']}Erro: Por favor, digite um número inteiro válido.{CORES['RESET']}")

    # Validação da Presença de Predadores
    while True:
        predadores_input = input("Há presença de predadores? (s/n): ").lower().strip()
        if predadores_input == 's':
            predadores = True
            break
        elif predadores_input == 'n':
            predadores = False
            break
        else:
            print(f"{CORES['VERMELHO']}Erro: Resposta inválida.{CORES['RESET']}")

    # Validação da Causa do Risco
    causas_validas = ["predador", "alagamento", "humano", "desconhecida", "nenhuma"]
    while True:
        causa_risco = input(f"Digite a causa do risco ({'/'.join(causas_validas)}): ").lower().strip()
        if causa_risco in causas_validas:
            break
        else:
            print(f"{CORES['VERMELHO']}Erro: Causa inválida.{CORES['RESET']}")

    novo_ninho = {
        "regiao": regiao,
        "quantidade_ovos": quantidade_ovos,
        "status": status,
        "risco": risco,
        "dias_para_eclosao": dias_para_eclosao,
        "predadores": predadores,
        "causa_risco": causa_risco
    }
    base_de_ninhos.append(novo_ninho)
    print(f"\n{CORES['VERDE']}---> Sucesso! Novo ninho registrado na base de dados. <---{CORES['RESET']}\n")
    desenhar_divisor("-")

def iniciar_sistema_avancado():
    """Função principal que inicia o loop do sistema com menu avançado."""
    while True:
        desenhar_divisor()
        print("*" + f"{CORES['NEGRITO']}{CORES['AZUL']} SISTEMA DE MONITORAMENTO: GUARDIÃO DAS TARTARUGUINHAS {CORES['RESET']}".center(98) + "*")
        desenhar_divisor()
        print("\nMENU PRINCIPAL:")
        print(f"  {CORES['AMARELO']}1. 🚨{CORES['RESET']} Painel de Alerta Rápido")
        print(f"  {CORES['CIANO']}2. 📜{CORES['RESET']} Relatório Completo")
        print(f"  {CORES['CIANO']}3. 📊{CORES['RESET']} Estatísticas Básicas")
        print(f"  {CORES['CIANO']}4. 📝{CORES['RESET']} Adicionar Novo Registro")
        print(f"  {CORES['CIANO']}5. 🔬{CORES['RESET']} Análise Avançada")
        print(f"  {CORES['CIANO']}6. 🚪{CORES['RESET']} Sair do Sistema")
        escolha = input("\n>> Digite a opção desejada: ").strip()
        if escolha == '1': exibir_painel_alerta(ninhos)
        elif escolha == '2': exibir_relatorio_completo(ninhos)
        elif escolha == '3': calcular_estatisticas_basicas(ninhos)
        elif escolha == '4': adicionar_novo_ninho(ninhos)
        elif escolha == '5':
            while True:
                print("\n--- Menu de Análise Avançada ---")
                print(f"  1. 📈 Análise Descritiva Completa")
                print(f"  2. 🔍 Análise Diagnóstica Cruzada")
                print(f"  3. ↩️  Voltar ao Menu Principal")
                sub_escolha = input("\n>> Escolha a análise: ").strip()
                if sub_escolha == '1': analise_descritiva_completa(ninhos)
                elif sub_escolha == '2': analise_diagnostica_cruzada(ninhos)
                elif sub_escolha == '3': break
                else: print(f"\n{CORES['VERMELHO']}[ERRO] Opção inválida.{CORES['RESET']}")
        elif escolha == '6':
            print(f"\n{CORES['VERDE']}Encerrando o sistema... Obrigado!{CORES['RESET']}\n")
            break
        else:
            print(f"\n{CORES['VERMELHO']}[ERRO] Opção inválida. Por favor, escolha um número de 1 a 6.{CORES['RESET']}")

# INICIALIZAÇÃO DO PROGRAMA
iniciar_sistema_avancado()


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*                SISTEMA DE MONITORAMENTO: GUARDIÃO DAS TARTARUGUINHAS                *
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MENU PRINCIPAL:
  1. 🚨 Painel de Alerta Rápido
  2. 📜 Relatório Completo
  3. 📊 Estatísticas Básicas
  4. 📝 Adicionar Novo Registro
  5. 🔬 Análise Avançada
  6. 🚪 Sair do Sistema

>> Digite a opção desejada: 1
||                            🚨 PAINEL DE ALERTA RÁPIDO 🚨                            ||

  - Ninhos em Risco Crítico (🔴): 3 ninhos
  - Eclosão Urgente (em até 48h): 2 ninhos
  - Região com Maior Risco Crítico: Praia Central
  - Taxa de Ninhos Intactos: 40.0%

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*                SISTEMA DE MONITORAMENTO: GUARDIÃO DAS TARTARUGUINHAS                *
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MENU PRINCIPAL:
  1. 🚨 Paine

KeyboardInterrupt: Interrupted by user